# Fase 1: Persiapan dan Pengayaan Data Spasial

In [7]:
import pandas as pd

# Path ke dataset dengan fitur temporal (hasil dari notebook 00_...)
path_fitur_temporal = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\parquet\dataset_final_features_energi_suhu.parquet'
df_temporal = pd.read_parquet(path_fitur_temporal)

# Path ke file koordinat yang Anda sediakan (ganti nama file jika perlu)
path_koordinat = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\meter_id\koordinat_gedung.xlsx' 

try:
    df_coords = pd.read_excel(path_koordinat) # Gunakan read_csv jika formatnya CSV
    print(f"Berhasil memuat data koordinat dengan {len(df_coords)} baris.")
    
    # --- Penggabungan Data (Langkah Krusial) ---
    # Menggabungkan berdasarkan 'meter_id' dari df_temporal dan 'gedung' dari df_coords
    df_spatial = pd.merge(
        df_temporal, 
        df_coords, 
        left_on='meter_id', 
        right_on='gedung', 
        how='left'
    )
    
    # Hapus kolom 'gedung' duplikat jika tidak diperlukan lagi
    df_spatial = df_spatial.drop(columns=['gedung'])
    
    # --- Validasi Menyeluruh ---
    print(f"\nUkuran DataFrame sebelum merge: {len(df_temporal)}")
    print(f"Ukuran DataFrame setelah merge: {len(df_spatial)}")
    assert len(df_temporal) == len(df_spatial), "Jumlah baris berubah setelah merge, ada yang salah!"

    # Verifikasi apakah ada meter_id yang tidak memiliki data koordinat
    if df_spatial['latitude'].isnull().any():
        print("\nPeringatan: Ditemukan 'meter_id' tanpa data koordinat:")
        gedung_tanpa_koordinat = df_spatial[df_spatial['latitude'].isnull()]['meter_id'].unique()
        print(gedung_tanpa_koordinat)
        # Opsi: Hapus baris-baris ini atau isi dengan nilai rata-rata (kurang direkomendasikan)
        # df_spatial.dropna(subset=['latitude'], inplace=True)
    else:
        print("\nSukses! Semua 'meter_id' berhasil digabungkan dengan data koordinat.")
        
    print("\nContoh data setelah digabungkan:")
    print(df_spatial[['timestamp', 'meter_id', 'konsumsi_energi', 'latitude', 'longitude']].head())

except FileNotFoundError:
    print(f"Error: File di path '{path_koordinat}' tidak ditemukan. Mohon periksa kembali path dan nama file.")
    df_spatial = None


Berhasil memuat data koordinat dengan 40 baris.

Ukuran DataFrame sebelum merge: 341440
Ukuran DataFrame setelah merge: 341440

Sukses! Semua 'meter_id' berhasil digabungkan dengan data koordinat.

Contoh data setelah digabungkan:
            timestamp meter_id  konsumsi_energi  latitude   longitude
0 2024-06-01 00:00:00    BSC A          22.6365 -6.891773  107.608495
1 2024-06-01 01:00:00    BSC A          22.8399 -6.891773  107.608495
2 2024-06-01 02:00:00    BSC A          22.5609 -6.891773  107.608495
3 2024-06-01 03:00:00    BSC A          22.5630 -6.891773  107.608495
4 2024-06-01 04:00:00    BSC A          22.3004 -6.891773  107.608495


# Fase 2: Rekayasa Fitur Spasial (Spatial Feature Engineering)

## 2.1. Metode Perhitungan Jarak

In [8]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Menghitung jarak (dalam kilometer) antara dua titik koordinat
    menggunakan formula Haversine.
    """
    R = 6371  # Radius rata-rata Bumi dalam kilometer
    
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance


## 2.2. Ide Fitur Spasial Baru (Pendekatan Bertingkat)

In [9]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Menghitung jarak (dalam kilometer) antara dua titik koordinat
    menggunakan formula Haversine.
    """
    R = 6371  # Radius rata-rata Bumi dalam kilometer
    
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance


In [10]:
# Definisikan titik-titik penting (Points of Interest)
POI = {
    'pusat_kampus': (-6.89036457, 107.61036290), # kolam indonesia tenggelam
    #'gardu_utama': (-6.8924, 107.6112)  # Misal Gardu PPTI
}

for poi_name, (lat, lon) in POI.items():
    df_spatial[f'jarak_ke_{poi_name}_km'] = df_spatial.apply(
        lambda row: haversine_distance(row['latitude'], row['longitude'], lat, lon),
        axis=1
    )


In [11]:
from sklearn.neighbors import NearestNeighbors

# --- Langkah A: Persiapan Data Tetangga ---
# Buat DataFrame unik berisi gedung dan koordinatnya untuk mencari tetangga
df_gedung_unique = df_coords[['gedung', 'latitude', 'longitude']].drop_duplicates().set_index('gedung')

# --- Langkah B: Menemukan Tetangga Berbasis K-Nearest Neighbors (KNN) ---
K = 4 # Kita cari 4 tetangga terdekat (karena tetangga ke-0 adalah gedung itu sendiri)
knn = NearestNeighbors(n_neighbors=K, metric='haversine')
knn.fit(np.radians(df_gedung_unique[['latitude', 'longitude']]))

# Dapatkan jarak dan indeks dari K tetangga terdekat
distances_knn, indices_knn = knn.kneighbors()

# Buat pemetaan (map) dari setiap gedung ke tetangga-tetangganya (kecuali dirinya sendiri)
# {gedung_A: [tetangga_1, tetangga_2, tetangga_3], ...}
neighbor_map_knn = {}
for i, gedung_nama in enumerate(df_gedung_unique.index):
    # Ambil indeks tetangga dari 1 sampai K (indeks 0 adalah gedung itu sendiri)
    neighbor_indices = indices_knn[i][1:]
    neighbor_map_knn[gedung_nama] = df_gedung_unique.index[neighbor_indices].tolist()

# --- Langkah C: Menemukan Tetangga Berbasis Radius ---
RADIUS_KM = 0.150 # 150 meter
neighbor_map_radius = {}
# Buat matriks jarak terlebih dahulu untuk efisiensi
gedung_list = df_gedung_unique.index
dist_matrix = pd.DataFrame(index=gedung_list, columns=gedung_list, dtype=float)
for g1 in gedung_list:
    for g2 in gedung_list:
        if g1 != g2:
            lat1, lon1 = df_gedung_unique.loc[g1]
            lat2, lon2 = df_gedung_unique.loc[g2]
            dist_matrix.loc[g1, g2] = haversine_distance(lat1, lon1, lat2, lon2)

for gedung_nama in gedung_list:
    # Temukan tetangga yang jaraknya <= RADIUS_KM
    tetangga_dalam_radius = dist_matrix.loc[gedung_nama][dist_matrix.loc[gedung_nama] <= RADIUS_KM].index.tolist()
    neighbor_map_radius[gedung_nama] = tetangga_dalam_radius

# --- Langkah D: Menghitung Fitur Spatial Lag (Cara Efisien) ---
print("\nMenghitung fitur spatial lag... Proses ini mungkin memakan waktu beberapa menit.")

# Buat kolom 'konsumsi_lag_1_jam' jika belum ada (sebagai contoh, kita asumsikan sudah ada)
if 'konsumsi_lag_1_jam' not in df_spatial.columns:
     df_spatial['konsumsi_lag_1_jam'] = df_spatial.groupby('meter_id')['konsumsi_energi'].shift(1)

# Pivot table untuk lookup nilai lag dengan cepat
pivot_lag = df_spatial.pivot_table(index='timestamp', columns='meter_id', values='konsumsi_lag_1_jam')

# Fungsi untuk menghitung rata-rata lag tetangga
def calculate_avg_neighbor_lag(row, neighbor_map):
    gedung = row['meter_id']
    timestamp = row['timestamp']
    tetangga = neighbor_map.get(gedung, [])

    # --- FIX ---
    # Cek apakah timestamp ada di index pivot table sebelum mencoba mengaksesnya.
    # Ini akan menangani kasus pada timestamp paling awal di mana semua lag adalah NaN,
    # sehingga timestamp tersebut tidak masuk ke dalam index pivot table.
    if timestamp not in pivot_lag.index:
        return np.nan
    # --- AKHIR DARI FIX ---

    if not tetangga:
        return np.nan # Tidak ada tetangga, kembalikan NaN

    # Ambil nilai lag dari tetangga pada timestamp yang sama dari pivot table
    neighbor_lags = pivot_lag.loc[timestamp, tetangga].values
    return np.nanmean(neighbor_lags) # Gunakan nanmean untuk mengabaikan NaN jika ada tetangga yang datanya kosong

# Terapkan fungsi untuk membuat fitur baru
df_spatial['avg_konsumsi_tetangga_knn3_lag1'] = df_spatial.apply(
    lambda row: calculate_avg_neighbor_lag(row, neighbor_map_knn), axis=1
)
df_spatial['avg_konsumsi_tetangga_150m_lag1'] = df_spatial.apply(
    lambda row: calculate_avg_neighbor_lag(row, neighbor_map_radius), axis=1
)

print("Selesai menghitung fitur spatial lag.")
print(df_spatial[['meter_id', 'avg_konsumsi_tetangga_knn3_lag1', 'avg_konsumsi_tetangga_150m_lag1']].head())

# --- Langkah E: Simpan Hasil Akhir ---
path_output_spatial = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\parquet\dataset_final_spatial_features.parquet'
df_spatial.to_parquet(path_output_spatial, index=False)
print(f"\nBerhasil! Dataset dengan fitur spasial telah disimpan ke:\n{path_output_spatial}")



Menghitung fitur spatial lag... Proses ini mungkin memakan waktu beberapa menit.
Selesai menghitung fitur spatial lag.
  meter_id  avg_konsumsi_tetangga_knn3_lag1  avg_konsumsi_tetangga_150m_lag1
0    BSC A                              NaN                              NaN
1    BSC A                         5.096806                         6.572714
2    BSC A                         5.114701                         6.569258
3    BSC A                         5.101916                         6.494553
4    BSC A                         5.097168                         6.485159

Berhasil! Dataset dengan fitur spasial telah disimpan ke:
C:\MyFolder\Git\TA_SpatioTemporal\Data\parquet\dataset_final_spatial_features.parquet
